IOB preparation

Source: Isabel Segura Bedmar

Exercise Spacy
https://colab.research.google.com/drive/1_PmpRHvjIePHmwAr1jFwmQ77sU0022kv


## Connect Drive


In [0]:
from google.colab import drive
drive.mount("/content/drive/")
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
drive  sample_data


# Import libraries

In [0]:
#dataframes
import pandas as pd
from tabulate import tabulate
import numpy as np


#regex
import re

#models
import spacy

In the notebook **"0904_terminado_Crear tsv a partir de text files.ipynb"**, there is a section called **"Create a tsv from all text files (train and dev)"**. It is necessary to execute it before load the independent text files.

https://colab.research.google.com/drive/1JhAqsmZBLJzpb0lA8BlU2Kzldv6WQ0b5#scrollTo=DLKJlTsxQ4YB


#Install Spacy and the model in spanish.



In [0]:
#pip install -U spacy #spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)   1 time
#!pip install spacy
!python -m spacy download es          #Every time it reload    ,   Successfully installed es-core-news-sm-2.2.5

import spacy
nlp = spacy.load('es')
#nlp = spacy.load("en_core_web_sm")  #Do es not work
print('spacy.es loaded')
  
!pip install spacy-lookup    #For Dictionary based NER ,  Successfully installed flashtext-2.7 spacy-lookup-0.1.0
from spacy_lookup import Entity

from spacy import displacy


     |████████████████████████████████| 16.2MB 578kB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172936 sha256=639aedfc6e3fcfaf5884c23d110e8bbb7480e1e1bc1f4b59501c9576cdf7e453
  Stored in directory: /tmp/pip-ephem-wheel-cache-i7kbvpnz/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/es_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/es
You can now load the model via spacy.load('es')
spacy.es loaded
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9299 sha256=8f696e5832c6b341e6df3af315d11bf335aea361f9e9421e598b7495f1906944
  Stored in directory: /root/.cache/pip/wheels/37/db/d7/fe74f7cb8e5c3afed90fe6f4967c933a6f13d81ab6b3d3128c
Successfully built flashtext


#Load data

1. Articles, Diagnostics/Procedures, train
2. Articles, Diagnostics/Procedures, dev
3. Codes, Diagnostics, train/dev
4. Annotations, Diagnostics, train
5. Annotations, Diagnostics, dev
6. Codes, Procedures, train/dev
7. Annotations, Procedures, train
8. Annotations, Procedures, dev

##Load articles from tsv (train and test)

In [0]:
def load_articles(path):
  df = pd.read_csv(path, sep=",", engine="python", encoding='utf-8')
  print("--- Loaded dataset:", path)

  print("--- Number of rows is " + str(df.shape[0]) + " x number of columns " + str(df.shape[1]))
  #print(df.head())
  #print(tabulate(df.head(), headers='keys', tablefmt='psql'))

  return df

path_codiesp='/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/'

print("- Load 1. Articles, Diagnostics/Procedures, train")
train_file_articles = path_codiesp + 'train_dev/train/train_articles_codiesp.csv'
df_train_articles = load_articles(train_file_articles)

print("- Load 2. Articles, Diagnostics/Procedures, dev")
dev_file_articles = path_codiesp + 'train_dev/dev/dev_articles_codiesp.csv'
df_dev_articles = load_articles(dev_file_articles)

- Load 1. Articles, Diagnostics/Procedures, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_articles_codiesp.csv
--- Number of rows is 500 x number of columns 2
- Load 2. Articles, Diagnostics/Procedures, dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/dev/dev_articles_codiesp.csv
--- Number of rows is 250 x number of columns 2


##Loading annotations and codes from tsv (train and dev, D and P)

In [0]:
def load_data(path, names):
  df = pd.read_csv(path, sep="\t", names=names)
  print("--- Loaded dataset:", path)

  print("--- Number of rows is " + str(df.shape[0]) + " x number of columns " + str(df.shape[1]))
  #print(df.head())
  #print(tabulate(df.head(), headers='keys', tablefmt='psql'))

  return df

path_codiesp='/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/'
names_annotations=["ArticleID", "Label", "ICD10-code", "Text-reference"]  #Columns: articleID \t label \t ICD10-code \t text-reference
names_codeD=["Code", "Es-description", "En-description"]   # Columns: code \t es-description \t en-description
names_codeP=["Code", "Es-description", "En-description", "Approach?"]   # Columns: code \t es-description \t en-description \t approach?

print("- Load 3. Codes, Diagnostics, train/dev")
file_codeD = path_codiesp + 'codiesp_codes/codiesp-D_codes.tsv'
df_codeD = load_data(file_codeD, names_codeD)

print("- Load 4. Annotations, Diagnostics, train")
file_train_annotationsD = path_codiesp + 'train_dev/train/train_annotations_codiespD_processed.tsv'
df_train_annotationsD = load_data(file_train_annotationsD, names_annotations)

print("- Load 5. Annotations, Diagnostics, dev")
file_dev_annotationsD = path_codiesp + 'train_dev/dev/dev_annotations_codiespD_processed.tsv'
df_dev_annotationsD = load_data(file_dev_annotationsD, names_annotations)

print("- Load 6. Codes, Procedures, train/dev")
file_codeP = path_codiesp + 'codiesp_codes/codiesp-P_codes.tsv'
df_codeP = load_data(file_codeP, names_codeP)

print("- Load 7. Annotations, Procedures, train")
file_train_annotationsP = path_codiesp + 'train_dev/train/train_annotations_codiespP_processed.tsv'
df_train_annotationsP = load_data(file_train_annotationsP, names_annotations)

print("- Load 8. Annotations, Procedures, dev")
file_dev_annotationsP = path_codiesp + 'train_dev/dev/dev_annotations_codiespP_processed.tsv'
df_dev_annotationsP = load_data(file_dev_annotationsP, names_annotations)

- Load 3. Codes, Diagnostics, train/dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/codiesp_codes/codiesp-D_codes.tsv
--- Number of rows is 71486 x number of columns 3
- Load 4. Annotations, Diagnostics, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_annotations_codiespD_processed.tsv
--- Number of rows is 7524 x number of columns 4
- Load 5. Annotations, Diagnostics, dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/dev/dev_annotations_codiespD_processed.tsv
--- Number of rows is 3589 x number of columns 4
- Load 6. Codes, Procedures, train/dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/codiesp_codes/codiesp-P_codes.tsv
--- Number of rows is 87170 x number of columns 4
- Load 7. Annotations, Procedures, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_annot

##Print dataframes for all the data

1. Articles, Diagnostics/Procedures, train
2. Articles, Diagnostics/Procedures, dev
3. Codes, Diagnostics, train/dev
4. Annotations, Diagnostics, train
5. Annotations, Diagnostics, dev
6. Codes, Procedures, train/dev
7. Annotations, Procedures, train
8. Annotations, Procedures, dev



In [0]:
print("- Print 1. Articles, Diagnostics/Procedures, train")
print(tabulate(df_train_articles.head(), headers='keys', tablefmt='psql'))

print("- Print 2. Articles, Diagnostics/Procedures, dev")
print(tabulate(df_dev_articles.head(), headers='keys', tablefmt='psql'))

print("- Print 3. Codes, Diagnostics, train/dev")
print(tabulate(df_codeD.head(), headers='keys', tablefmt='psql'))

print("- Print 4. Annotations, Diagnostics, train")
print(tabulate(df_train_annotationsD.head(), headers='keys', tablefmt='psql'))

print("- Print 5. Annotations, Diagnostics, dev")
print(tabulate(df_dev_annotationsD.head(), headers='keys', tablefmt='psql'))

print("- Print 6. Codes, Procedures, train/dev")
print(tabulate(df_codeP.head(), headers='keys', tablefmt='psql')) 

print("- Print 7. Annotations, Procedures, train")
print(tabulate(df_train_annotationsP.head(), headers='keys', tablefmt='psql'))   

print("- Print 8. Annotations, Procedures, dev")
print(tabulate(df_dev_annotationsP.head(), headers='keys', tablefmt='psql'))

- Print 1. Articles, Diagnostics/Procedures, train
+----+---------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Prepare IOB and features


Features:
-	token

- token_PoS

- token_IOB

- orth

- lemma

- shape

- isUpper '

- isNumber '

- isTitle 

- isLower

- isPunct

- start

-	end

-	w-2
-	w-2_PoS
-	w-2_IOB

-	w-1
-	w-1_PoS
-	w-1_IOB

-	w1
- w1_PoS	
- w1_IOB

-	w2
-	w2_PoS
-	w2_IOB



##Trial with 1 text file (train)

In [0]:
"""Loop for one text file: S0004-06142005000700014-1 is called text, wich is going to de document"""

for index, row in df_train_articles.iterrows():
    if index == 0:
      text_train = row[1]
      print(row[0], text_train)
    elif index ==1:
      text1_train = row[1]
      print(row[0], text1_train)

#text = text.lower() #en el caso de poner todo minusculas

S0004-06142005000700014-1 Describimos el caso de un varón de 37 años con vida previa activa que refiere dolores osteoarticulares de localización variable en el último mes y fiebre en la última semana con picos (matutino y vespertino) de 40 C las últimas 24-48 horas, por lo que acude al Servicio de Urgencias. Antes de comenzar el cuadro estuvo en Extremadura en una región endémica de brucella, ingiriendo leche de cabra sin pasteurizar y queso de dicho ganado. Entre los comensales aparecieron varios casos de brucelosis. Durante el ingreso para estudio del síndrome febril con antecedentes epidemiológicos de posible exposición a Brucella presenta un cuadro de orquiepididimitis derecha.
La exploración física revela: Tª 40,2 C; T.A: 109/68 mmHg; Fc: 105 lpm. Se encuentra consciente, orientado, sudoroso, eupneico, con buen estado de nutrición e hidratación. En cabeza y cuello no se palpan adenopatías, ni bocio ni ingurgitación de vena yugular, con pulsos carotídeos simétricos. Auscultación ca

In [0]:
"""TEXT FILE S0004-06142005000700014-1 :
Describimos el caso de un varón de 37 años con vida previa activa que refiere dolores osteoarticulares de localización variable en el último mes y fiebre en la última semana con picos (matutino y vespertino) de 40 C las últimas 24-48 horas, por lo que acude al Servicio de Urgencias. Antes de comenzar el cuadro estuvo en Extremadura en una región endémica de brucella, ingiriendo leche de cabra sin pasteurizar y queso de dicho ganado. Entre los comensales aparecieron varios casos de brucelosis. Durante el ingreso para estudio del síndrome febril con antecedentes epidemiológicos de posible exposición a Brucella presenta un cuadro de orquiepididimitis derecha.
La exploración física revela: Tª 40,2 C; T.A: 109/68 mmHg; Fc: 105 lpm. Se encuentra consciente, orientado, sudoroso, eupneico, con buen estado de nutrición e hidratación. En cabeza y cuello no se palpan adenopatías, ni bocio ni ingurgitación de vena yugular, con pulsos carotídeos simétricos. Auscultación cardíaca rítmica, sin soplos, roces ni extratonos. Auscultación pulmonar con conservación del murmullo vesicular. Abdomen blando, depresible, sin masas ni megalias. En la exploración neurológica no se detectan signos meníngeos ni datos de focalidad. Extremidades sin varices ni edemas. Pulsos periféricos presentes y simétricos. En la exploración urológica se aprecia el teste derecho aumentado de tamaño, no adherido a piel, con zonas de fluctuación e intensamente doloroso a la palpación, con pérdida del límite epidídimo-testicular y transiluminación positiva.
Los datos analíticos muestran los siguentes resultados: Hemograma: Hb 13,7 g/dl; leucocitos 14.610/mm3 (neutrófilos 77%); plaquetas 206.000/ mm3. VSG: 40 mm 1ª hora. Coagulación: TQ 87%; TTPA 25,8 seg. Bioquímica: Glucosa 117 mg/dl; urea 29 mg/dl; creatinina 0,9 mg/dl; sodio 136 mEq/l; potasio 3,6 mEq/l; GOT 11 U/l; GPT 24 U/l; GGT 34 U/l; fosfatasa alcalina 136 U/l; calcio 8,3 mg/dl. Orina: sedimento normal.
Durante el ingreso se solicitan Hemocultivos: positivo para Brucella y Serologías específicas para Brucella: Rosa de Bengala +++; Test de Coombs > 1/1280; Brucellacapt > 1/5120. Las pruebas de imagen solicitadas ( Rx tórax, Ecografía abdominal, TAC craneal, Ecocardiograma transtorácico) no evidencian patología significativa, excepto la Ecografía testicular, que muestra engrosamiento de la bolsa escrotal con pequeña cantidad de líquido con septos y testículo aumentado de tamaño con pequeñas zonas hipoecoicas en su interior que pueden representar microabscesos.
Con el diagnóstico de orquiepididimitis secundaria a Brucella se instaura tratamiento sintomático (antitérmicos, antiinflamatorios, reposo y elevación testicular) así como tratamiento antibiótico específico: Doxiciclina 100 mg vía oral cada 12 horas (durante 6 semanas) y Estreptomicina 1 gramo intramuscular cada 24 horas (durante 3 semanas). El paciente mejora significativamente de su cuadro tras una semana de ingreso, decidiéndose el alta a su domicilio donde completó la pauta de tratamiento antibiótico. En revisiones sucesivas en consultas se constató la completa remisión del cuadro.
"""

"""DIAGNOSTICS
|    0 | S0004-06142005000700014-1  | DIAGNOSTICO | e04.9 (automatic suggestion) | bocio                                                                                                             |
|    1 | S0004-06142005000700014-1  | DIAGNOSTICO | a23.9                        | Brucella                                                                                                          |
|    2 | S0004-06142005000700014-1  | DIAGNOSTICO | a23.9                        | Brucella                                                                                                          |
|    3 | S0004-06142005000700014-1  | DIAGNOSTICO | r50.9                        | febril                                                                                                            |
|    4 | S0004-06142005000700014-1  | DIAGNOSTICO | r52                          | dolores                                                                                                           |
|    5 | S0004-06142005000700014-1  | DIAGNOSTICO | r60.9                        | edemas                                                                                                            |
|    6 | S0004-06142005000700014-1  | DIAGNOSTICO | z20.818                      | exposición a Brucella                                                                                             |
|    7 | S0004-06142005000700014-1  | DIAGNOSTICO | a23.9                        | Brucella                                                                                                          |
|    8 | S0004-06142005000700014-1  | DIAGNOSTICO | n44.8                        | teste derecho aumentado de tamaño                                                                                 |
|    9 | S0004-06142005000700014-1  | DIAGNOSTICO | i87.8                        | ingurgitación de vena yugular                                                                                     |
|   10 | S0004-06142005000700014-1  | DIAGNOSTICO | a23.9                        | brucella                                                                                                          |
|   11 | S0004-06142005000700014-1  | DIAGNOSTICO | n45.3                        | orquiepididimitis                                                                                                 |
|   12 | S0004-06142005000700014-1  | DIAGNOSTICO | n45.3                        | orquiepididimitis                                                                                                 |
|   13 | S0004-06142005000700014-1  | DIAGNOSTICO | i83.90                       | varices                                                                                                           |
|   14 | S0004-06142005000700014-1  | DIAGNOSTICO | m25.50                       | dolores osteoarticulares                                                                                          |
|   15 | S0004-06142005000700014-1  | DIAGNOSTICO | r50.9                        | fiebre                                                                                                            |
|   16 | S0004-06142005000700014-1  | DIAGNOSTICO | r50.9                        | síndrome febril      


PROCEDURES
|    0 | S0004-06142005000700014-1  | PROCEDIMIENTO | bw03zzz                        | Rx tórax                                                                                                                                |
|    1 | S0004-06142005000700014-1  | PROCEDIMIENTO | bn20                           | TAC craneal                                                                                                                             |
|    2 | S0004-06142005000700014-1  | PROCEDIMIENTO | bw40zzz                        | Ecografía abdominal                                                                                                                     |
|    3 | S0004-06142005000700014-1  | PROCEDIMIENTO | bv44zzz                        | Ecografía testicular                                                                                                                    |
|    4 | S0004-06142005000700014-1  | PROCEDIMIENTO | 3E02329                        | Estreptomicina intramuscular"""  


D {'bocio': 'B', 'Brucella': 'B', 'febril': 'I', 'dolores': 'B', 'edemas': 'B', 'exposición': 'B', 'a': 'I', 'teste': 'B', 'derecho': 'I', 'aumentado': 'I', 'de': 'I', 'tamaño': 'I', 'ingurgitación': 'B', 'vena': 'I', 'yugular': 'I', 'brucella': 'B', 'orquiepididimitis': 'B', 'varices': 'B', 'osteoarticulares': 'I', 'fiebre': 'B', 'síndrome': 'B'}


'DIAGNOSTICS\n|    0 | S0004-06142005000700014-1  | DIAGNOSTICO | e04.9 (automatic suggestion) | bocio                                                                                                             |\n|    1 | S0004-06142005000700014-1  | DIAGNOSTICO | a23.9                        | Brucella                                                                                                          |\n|    2 | S0004-06142005000700014-1  | DIAGNOSTICO | a23.9                        | Brucella                                                                                                          |\n|    3 | S0004-06142005000700014-1  | DIAGNOSTICO | r50.9                        | febril                                                                                                            |\n|    4 | S0004-06142005000700014-1  | DIAGNOSTICO | r52                          | dolores                                                                                                

In [0]:
def Save_list_entities(df, tag):
  # a dict with the entitites of each abstract classifying them into 'B' or 'I'
  entities = {}

  for index, row in df.iterrows():
      article_id = row[0]
      #print(article_id)
      entity= row[3].split()
      #print(entity)
      for j, ent in enumerate(entity):
        if j == 0:
          entities.setdefault(article_id, {})[ent] =  'B-'+str(tag)
        else:
          entities.setdefault(article_id, {})[ent] =  'I-'+str(tag)
  print(entities)
  return(entities)

tag_diagnostic = "D"
tag_procedure ="P"
entities_D =  Save_list_entities(df_train_annotationsD, tag_diagnostic)
entities_P =  Save_list_entities(df_train_annotationsP, tag_procedure)


{'S0004-06142005000700014-1': {'bocio': 'B-D', 'Brucella': 'B-D', 'febril': 'I-D', 'dolores': 'B-D', 'edemas': 'B-D', 'exposición': 'B-D', 'a': 'I-D', 'teste': 'B-D', 'derecho': 'I-D', 'aumentado': 'I-D', 'de': 'I-D', 'tamaño': 'I-D', 'ingurgitación': 'B-D', 'vena': 'I-D', 'yugular': 'I-D', 'brucella': 'B-D', 'orquiepididimitis': 'B-D', 'varices': 'B-D', 'osteoarticulares': 'I-D', 'fiebre': 'B-D', 'síndrome': 'B-D'}, 'S0004-06142005000900013-1': {'leiomioma': 'B-D', 'vesical': 'I-D', 'sangrado': 'B-D'}, 'S0004-06142005000900015-1': {'rabdomiosarcoma': 'B-D', 'pleomórfico': 'I-D', 'del': 'I-D', 'cordón': 'B-D', 'espermático': 'I-D', 'pérdida': 'B-D', 'de': 'I-D', 'peso': 'I-D', 'dolorosa': 'B-D', 'tumor': 'B-D', 'paratesticular': 'I-D', 'izquierdo': 'I-D', 'tumorales': 'B-D', 'neoplásica': 'I-D', 'retroperitoneal': 'B-D', 'masa': 'B-D', 'doloroso': 'I-D', 'testes': 'I-D'}, 'S0004-06142005001000011-3': {'porta': 'B-D', 'sonda': 'I-D', 'vesical': 'I-D', 'paraparesia': 'B-D', 'espástica': 

In [0]:
article_id = "S0004-06142005000700014-1"                    #the first text file to try
#article_id = "S0465-546X2009000300008-1"                   #one file without procedure entities
                 

if article_id in entities_D.keys():
  print(entities_D[article_id])
else:
  print("None")


if article_id in entities_P.keys():
  print(entities_P[article_id])
else:
  print("None")


{'bocio': 'B-D', 'Brucella': 'B-D', 'febril': 'I-D', 'dolores': 'B-D', 'edemas': 'B-D', 'exposición': 'B-D', 'a': 'I-D', 'teste': 'B-D', 'derecho': 'I-D', 'aumentado': 'I-D', 'de': 'I-D', 'tamaño': 'I-D', 'ingurgitación': 'B-D', 'vena': 'I-D', 'yugular': 'I-D', 'brucella': 'B-D', 'orquiepididimitis': 'B-D', 'varices': 'B-D', 'osteoarticulares': 'I-D', 'fiebre': 'B-D', 'síndrome': 'B-D'}
{'Rx': 'B-P', 'tórax': 'I-P', 'TAC': 'B-P', 'craneal': 'I-P', 'Ecografía': 'B-P', 'abdominal': 'I-P', 'testicular': 'I-P', 'Estreptomicina': 'B-P', 'intramuscular': 'I-P'}


In [0]:
def Tag_entities(document):
  sentence_index_list = []
  token_list = []
  token_pos_list = []
  token_iob_d_list = []
  token_iob_p_list = []


  #print(text_train)
  document = nlp(text_train)

  for k, s in enumerate(document.sents):                      #divide document into sentences
          #print("--------------------------------------------------------")
          print("sentence index: ", k, "\tsentence: ", s)
          #print(s.start_char)                     #start_char and end_char are the position of the first and last character of sentence respecting the document.
          #print(s.end_char)
          for i, token in enumerate(s):
            if i==0:
              sentence_index_list.append(i)
            else:
              sentence_index_list.append("NaN")

            print("\ttoken index: ", i, "\ttoken: ", token)
            token_list.append(token.text)

            #print("\ttoken_PoS: ", token.pos_)
            token_pos_list.append(token.pos_)

            #print("\ttoken orth: ", token.orth_)
            #print("\ttoken lemma: ", token.lemma_)
            #print("\ttoken shape: ", token.shape_)
            #print(\ttoken is upper: ", str(token.is_upper))
            #print(\ttoken is number: ", str(token.is_digit))
            #print(\ttoken is title: ", str(token.is_title))         
            #print(\ttoken is lower: ", str(token.is_lower))
            #print(\ttoken is punctuation: ", str(token.is_punct))

            # the start character of the sentence is subtracted from the token offset in the text to get the offset of the token in the sentence
            sentence_start_char = token.idx - s.start_char
            #print(str(sentence_start_char))
            #print(str(sentence_start_char + len(token.text) - 1))


            #print(article_id)
            if article_id in entities_D.keys():                                    ###Check if there are entities D in that file
              #print(entities_D[article_id])

              if token.text in entities_D[article_id]:                           ###IOB tagging D for token.  if the token is in the entities of the article_id get its class (I or B), else O
                ##print("\t\ttoken: ", token.text)
                ##print("\t\ttoken_IOB is : ", entities_D[article_id][token.text])
                token_iob_d_list.append(entities_D[article_id][token.text])      
              else:
                ##print("\t\ttoken_IOB is : O")
                token_iob_d_list.append("O")
            else:
              #print("None D entities")
              token_iob_d_list.append("O")
            

            
            if article_id in entities_P.keys():                                ###Check if there are entities P in that file
              #print(entities_P[article_id])
              
              if token.text in entities_P[article_id]:                         ###IOB tagging P for token. if the token is in the entities of the article_id get its class (I or B), else O
                ##print("\t\ttoken: ", token.text)
                ##print("\t\ttoken_IOB is : ", entities_P[article_id][token.text])
                token_iob_p_list.append(entities_P[article_id][token.text])          
              else:
                ##print("\t\ttoken_IOB is : O")
                token_iob_p_list.append("O")          
            else:
              #print("None P entities")
              token_iob_p_list.append("O")




            ###IOB tagging  for words around the token (word-2, word-1, word+1 and word+2)
            #for j in range(-2, 3):
              #if j != 0:
                #if 0 <= i + j < len(s):
                  #print("\t\tword", j, ": ", s[i + j].text)
                  #print("\t\tword", j, "_PoS: ", s[i + j].pos_ + "\t")


                  ###IOB tagging D for token
                  #if s[i + j].text in entities_d[article_id]:
                    #print("\t\tword", j , "_IOB: ", entities_d[article_id][s[i + j].text])
                  #else:
                    #print("\t\tword", j, "_IOB: O")

                  ###IOB tagging P for token
                  #if s[i + j].text in entities_p[article_id]:
                    #print("\t\tword", j , "_IOB: ", entities_p[article_id][s[i + j].text])
                  #else:
                    #print("\t\tword", j, "_IOB: O")
  return(sentence_index_list, token_list, token_pos_list, token_iob_d_list, token_iob_p_list)



sentence_index_list, token_list, token_pos_list, token_iob_d_list, token_iob_p_list = Tag_entities(document)

sentence index:  0 	sentence:  Describimos el caso de un varón de 37 años con vida previa activa que refiere dolores osteoarticulares de localización variable en el último mes y fiebre en la última semana con picos (matutino y vespertino) de 40 C las últimas 24-48 horas, por lo que acude al Servicio de Urgencias.
	token index:  0 	token:  Describimos
	token index:  1 	token:  el
	token index:  2 	token:  caso
	token index:  3 	token:  de
	token index:  4 	token:  un
	token index:  5 	token:  varón
	token index:  6 	token:  de
	token index:  7 	token:  37
	token index:  8 	token:  años
	token index:  9 	token:  con
	token index:  10 	token:  vida
	token index:  11 	token:  previa
	token index:  12 	token:  activa
	token index:  13 	token:  que
	token index:  14 	token:  refiere
	token index:  15 	token:  dolores
	token index:  16 	token:  osteoarticulares
	token index:  17 	token:  de
	token index:  18 	token:  localización
	token index:  19 	token:  variable
	token index:  20 	token:  

In [0]:
def Create_transposed_list(list):
  print("lenght and list are: ", len(list), "\t", list)
  np_list = np.asarray([list]).T
  #print("lenght and transposed token IOB list are: ", len(np_list), "\t", np_list)
  return(np_list)

np_sentence_index_list = Create_transposed_list(sentence_index_list)
np_token_list = Create_transposed_list(token_list)
np_token_pos_list = Create_transposed_list(token_pos_list)
np_token_iob_d_list = Create_transposed_list(token_iob_d_list)
np_token_iob_p_list = Create_transposed_list(token_iob_p_list)

lenght and list are:  576 	 [0, 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 0, 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 0, 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 0, 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 0, 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 0, 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', '

In [0]:
def Create_iob_dataframe(token_list, np_sentence_index_list, np_token_list, np_token_pos_list, np_token_iob_d_list, np_token_iob_p_list):  
  df = pd.DataFrame(columns=['Sentence #','token', 'token_PoS', 'token_IOB-D', 'token_IOB-P'])
  for a in range(len(token_list)):
    df.loc[a] = [np.random.randint(0,2) for n in range(5)]
  for b in range(len(token_list)):
    df.loc[:, ['Sentence #']] = np_sentence_index_list
    df.loc[:, ['token']] = np_token_list
    df.loc[:, ['token_PoS']] = np_token_pos_list
    df.loc[:, ['token_IOB-D']] = np_token_iob_d_list
    df.loc[:, ['token_IOB-P']] = np_token_iob_p_list

  print(tabulate(df, headers= 'keys', tablefmt='psql'))
  return(df)

print("--- Dataframed IOB train, Text-file: ", article_id)    
df = Create_iob_dataframe(token_list, np_sentence_index_list, np_token_list, np_token_pos_list, np_token_iob_d_list, np_token_iob_p_list)

--- Dataframed IOB train, Text-file:  S0004-06142005000700014-1
+-----+--------------+--------------------+-------------+---------------+---------------+
|     |   Sentence # | token              | token_PoS   | token_IOB-D   | token_IOB-P   |
|-----+--------------+--------------------+-------------+---------------+---------------|
|   0 |            0 | Describimos        | VERB        | O             | O             |
|   1 |          nan | el                 | DET         | O             | O             |
|   2 |          nan | caso               | NOUN        | O             | O             |
|   3 |          nan | de                 | ADP         | I-D           | O             |
|   4 |          nan | un                 | DET         | O             | O             |
|   5 |          nan | varón              | NOUN        | O             | O             |
|   6 |          nan | de                 | ADP         | I-D           | O             |
|   7 |          nan | 37           

##Trial with all text files (train)

In [0]:
def Save_list_entities(df, tag):
  # a dict with the entitites of each abstract classifying them into 'B' or 'I'
  entities = {}

  for index, row in df.iterrows():
      article_id = row[0]
      entity= row[3].split()
      #print(entity)
      for j, ent in enumerate(entity):
        if j == 0:
          entities.setdefault(article_id, {})[ent] =  'B-'+str(tag)
        else:
          entities.setdefault(article_id, {})[ent] =  'I-'+str(tag)
  ##print(entities)
  return(entities)

def Tag_entities(document):
  sentence_index_list = []
  token_list = []
  token_pos_list = []
  token_iob_d_list = []
  token_iob_p_list = []
  for k, s in enumerate(document.sents):                      #divide document into sentences
          #print("--------------------------------------------------------")
          ##print("sentence index: ", k, "\tsentence: ", s)
          #print(s.start_char)                     #start_char and end_char are the position of the first and last character of sentence respecting the document.
          #print(s.end_char)
          for i, token in enumerate(s):
            if i==0:
              sentence_index_list.append(k)
            else:
              sentence_index_list.append("NaN")

            ##print("\ttoken index: ", i, "\ttoken: ", token)
            token_list.append(token.text)

            #print("\ttoken_PoS: ", token.pos_)
            token_pos_list.append(token.pos_)

            #print("\ttoken orth: ", token.orth_)
            #print("\ttoken lemma: ", token.lemma_)
            #print("\ttoken shape: ", token.shape_)
            #print(\ttoken is upper: ", str(token.is_upper))
            #print(\ttoken is number: ", str(token.is_digit))
            #print(\ttoken is title: ", str(token.is_title))         
            #print(\ttoken is lower: ", str(token.is_lower))
            #print(\ttoken is punctuation: ", str(token.is_punct))

            # the start character of the sentence is subtracted from the token offset in the text to get the offset of the token in the sentence
            sentence_start_char = token.idx - s.start_char
            #print(str(sentence_start_char))
            #print(str(sentence_start_char + len(token.text) - 1))

            
            #print(article_id)
            if article_id in entities_D.keys():                                    ###Check if there are entities D in that file
              #print(entities_D[article_id])

              if token.text in entities_D[article_id]:                           ###IOB tagging D for token.  if the token is in the entities of the article_id get its class (I or B), else O
                ##print("\t\ttoken: ", token.text)
                ##print("\t\ttoken_IOB is : ", entities_D[article_id][token.text])
                token_iob_d_list.append(entities_D[article_id][token.text])      
              else:
                ##print("\t\ttoken_IOB is : O")
                token_iob_d_list.append("O")
            else:
              #print("None D entities")
              token_iob_d_list.append("O")
            

            
            if article_id in entities_P.keys():                                ###Check if there are entities P in that file
              #print(entities_P[article_id])
              
              if token.text in entities_P[article_id]:                         ###IOB tagging P for token. if the token is in the entities of the article_id get its class (I or B), else O
                ##print("\t\ttoken: ", token.text)
                ##print("\t\ttoken_IOB is : ", entities_P[article_id][token.text])
                token_iob_p_list.append(entities_P[article_id][token.text])          
              else:
                ##print("\t\ttoken_IOB is : O")
                token_iob_p_list.append("O")          
            else:
              #print("None P entities")
              token_iob_p_list.append("O")




            ###IOB tagging  for words around the token (word-2, word-1, word+1 and word+2)
            #for j in range(-2, 3):
              #if j != 0:
                #if 0 <= i + j < len(s):
                  #print("\t\tword", j, ": ", s[i + j].text)
                  #print("\t\tword", j, "_PoS: ", s[i + j].pos_ + "\t")


                  ###IOB tagging D for token
                  #if s[i + j].text in entities_d[article_id]:
                    #print("\t\tword", j , "_IOB: ", entities_d[article_id][s[i + j].text])
                  #else:
                    #print("\t\tword", j, "_IOB: O")

                  ###IOB tagging P for token
                  #if s[i + j].text in entities_p[article_id]:
                    #print("\t\tword", j , "_IOB: ", entities_p[article_id][s[i + j].text])
                  #else:
                    #print("\t\tword", j, "_IOB: O")
  return(sentence_index_list, token_list, token_pos_list, token_iob_d_list, token_iob_p_list)

def Create_transposed_list(list):
  ##print("lenght and list are: ", len(list), "\t", list)
  np_list = np.asarray([list]).T
  #print("lenght and transposed token IOB list are: ", len(np_list), "\t", np_list)
  return(np_list)

def Create_iob_dataframe(token_list, np_sentence_index_list, np_token_list, np_token_pos_list, np_token_iob_d_list, np_token_iob_p_list):  
  df = pd.DataFrame(columns=['Sentence #','token', 'token_PoS', 'token_IOB-D', 'token_IOB-P'])
  for a in range(len(token_list)):
    df.loc[a] = [np.random.randint(0,2) for n in range(5)]
  for b in range(len(token_list)):
    df.loc[:, ['Sentence #']] = np_sentence_index_list
    df.loc[:, ['token']] = np_token_list
    df.loc[:, ['token_PoS']] = np_token_pos_list
    df.loc[:, ['token_IOB-D']] = np_token_iob_d_list
    df.loc[:, ['token_IOB-P']] = np_token_iob_p_list

  #print(tabulate(df, headers= 'keys', tablefmt='psql'))
  return(df)

In [0]:
tag_diagnostic = "D"
tag_procedure ="P"
entities_D =  Save_list_entities(df_train_annotationsD, tag_diagnostic)
entities_P =  Save_list_entities(df_train_annotationsP, tag_procedure)

for index, row in df_train_articles.iterrows():
    text_train = row[1]
    article_id = str(row[0])
    #print(article_id)
    #if article_id in entities_D.keys():
      #print(entities_D[article_id])
    #else:
      #print("None D entities")
    #if article_id in entities_P.keys():
      #print(entities_P[article_id])
    #else:
      #print("None P entities")

    document = nlp(text_train)
    sentence_index_list, token_list, token_pos_list, token_iob_d_list, token_iob_p_list = Tag_entities(document)
    
    np_sentence_index_list = Create_transposed_list(sentence_index_list)
    np_token_list = Create_transposed_list(token_list)
    #print(token_list)
    #print(len(token_list))
    #print(len(sentence_index_list))
    #print(len(token_pos_list))
    #print(len(token_iob_d_list))
    #print(len(token_iob_p_list))

    np_token_pos_list = Create_transposed_list(token_pos_list)
    np_token_iob_d_list = Create_transposed_list(token_iob_d_list)
    np_token_iob_p_list = Create_transposed_list(token_iob_p_list)
    print("--- Dataframed IOB train, Text-file: ", article_id)    
    df = Create_iob_dataframe(token_list, np_sentence_index_list, np_token_list, np_token_pos_list, np_token_iob_d_list, np_token_iob_p_list)

    path = '/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/text_files_iob/train_iob_' + str(article_id) + '.csv'
    #print(path)
    df.to_csv(path, index=False) #guardar dataframe como tsv sin el indice de la izquierda
    print("---Saved IOB train, Text-file: ", article_id) 

--- Dataframed IOB train, Text-file:  S0004-06142005000700014-1
---Saved IOB train, Text-file:  S0004-06142005000700014-1
--- Dataframed IOB train, Text-file:  S0378-48352006000600006-1
---Saved IOB train, Text-file:  S0378-48352006000600006-1
--- Dataframed IOB train, Text-file:  S0004-06142005000900013-1
---Saved IOB train, Text-file:  S0004-06142005000900013-1
--- Dataframed IOB train, Text-file:  S0378-48352006000900006-1
---Saved IOB train, Text-file:  S0378-48352006000900006-1
--- Dataframed IOB train, Text-file:  S0004-06142005000900015-1
---Saved IOB train, Text-file:  S0004-06142005000900015-1
--- Dataframed IOB train, Text-file:  S0465-546X2008000200010-1
---Saved IOB train, Text-file:  S0465-546X2008000200010-1
--- Dataframed IOB train, Text-file:  S0004-06142005001000011-3
---Saved IOB train, Text-file:  S0004-06142005001000011-3
--- Dataframed IOB train, Text-file:  S0465-546X2009000300008-1
---Saved IOB train, Text-file:  S0465-546X2009000300008-1
--- Dataframed IOB train

##Old incomplete without functions for 1 text

In [0]:
# a dict with the entitites of each abstract classifying them into 'B' or 'I'
entities = {}

for index, row in df_train_annotationsD.iterrows():
    article_id = row[0]
    entity= row[3].split()
    #print(entity)
    for j, ent in enumerate(entity):
      if j == 0:
        entities.setdefault(article_id, {})[ent] =  'B-D'
      else:
        entities.setdefault(article_id, {})[ent] =  'I-D'
print(entities)


article_id = "S0004-06142005000700014-1"
print(entities[article_id])




sentence_index_list = []
token_list = []
token_pos_list = []
token_iob_d_list = []




document = nlp(text_train)

#print(text_train)
document = nlp(text_train)

for k, s in enumerate(document.sents):                      #divide document into sentences
        #print("--------------------------------------------------------")
        print("sentence index: ", k, "\tsentence: ", s)
        #print(s.start_char)                     #start_char and end_char are the position of the first and last character of sentence respecting the document.
        #print(s.end_char)
        for i, token in enumerate(s):
          if i==0:
            sentence_index_list.append(i)
          else:
            sentence_index_list.append("NaN")

          print("\ttoken index: ", i, "\ttoken: ", token)
          token_list.append(token.text)

          #print("\ttoken_PoS: ", token.pos_)
          token_pos_list.append(token.pos_)

          #print("\ttoken orth: ", token.orth_)
          #print("\ttoken lemma: ", token.lemma_)
          #print("\ttoken shape: ", token.shape_)
          #print(\ttoken is upper: ", str(token.is_upper))
          #print(\ttoken is number: ", str(token.is_digit))
          #print(\ttoken is title: ", str(token.is_title))         
          #print(\ttoken is lower: ", str(token.is_lower))
          #print(\ttoken is punctuation: ", str(token.is_punct))

          # the start character of the sentence is subtracted from the token offset in the text to get the offset of the token in the sentence
          sentence_start_char = token.idx - s.start_char
          #print(str(sentence_start_char))
          #print(str(sentence_start_char + len(token.text) - 1))


          ###IOB tagging for token
          if token.text in entities[article_id]:                         #if the token is in the entities of the article_id get its class (I or B), else O
            print("\t\ttoken: ", token.text)
            print("\t\ttoken_IOB is : ", entities[article_id][token.text])
            token_iob_d_list.append(entities[article_id][token.text])
          else:
            print("\t\ttoken_IOB is : O")
            token_iob_d_list.append("O")


          ###IOB tagging for words around the token (word-2, word-1, word+1 and word+2)
          #for j in range(-2, 3):
            #if j != 0:
              #if 0 <= i + j < len(s):
                #print("\t\tword", j, ": ", s[i + j].text)
                #print("\t\tword", j, "_PoS: ", s[i + j].pos_ + "\t")
                #if s[i + j].text in entities[article_id]:
                  #print("\t\tword", j , "_IOB: ", entities[article_id][s[i + j].text])
                #else:
                  #print("\t\tword", j, "_IOB: O")


print("lenght and sentence index list are: ", len(sentence_index_list), "\t", sentence_index_list)
np_sentence_index_list = np.asarray([sentence_index_list]).T
#print("lenght and transposed sentence index list are: ", len(np_sentence_index_list), "\t", np_sentence_index_list)

print("lenght and token list are: ", len(token_list), "\t", token_list)
np_token_list = np.asarray([token_list]).T
#print("lenght and transposed token list are: ", len(np_token_list), "\t", np_token_list)

print("lenght and token PoS list are: ", len(token_pos_list), "\t", token_pos_list)
np_token_pos_list = np.asarray([token_pos_list]).T
#print("lenght and transposed token PoS list are: ", len(np_token_pos_list), "\t", np_token_pos_list)

print("lenght and token IOB D list are: ", len(token_iob_d_list), "\t", token_iob_d_list)
np_token_iob_d_list = np.asarray([token_iob_d_list]).T
#print("lenght and transposed token IOB list are: ", len(np_token_iob_d_list), "\t", np_token_iob_d_list)

print("lenght and token IOB P list are: ", len(token_iob_p_list), "\t", token_iob_list)
np_token_iob_p_list = np.asarray([token_iob_list]).T
#print("lenght and transposed token IOB list are: ", len(np_token_iob_p_list), "\t", np_token_iob_p_list)


df = pd.DataFrame(columns=['Sentence #','token', 'token_PoS', 'token_IOB-D', 'token_IOB-P'])

for a in range(len(token_list)):
  df.loc[a] = [np.random.randint(0,2) for n in range(5)]
for b in range(len(token_list)):
  df.loc[:, ['Sentence #']] = np_sentence_index_list
  df.loc[:, ['token']] = np_token_list
  df.loc[:, ['token_PoS']] = np_token_pos_list
  df.loc[:, ['token_IOB-D']] = np_token_iob_d_list
  df.loc[:, ['token_IOB-P']] = np_token_iob_p_list

print(tabulate(df, headers= 'keys', tablefmt='psql'))
#df_features_text_train = df

#Output

All the text_files with: Sentence index, Word, PoS, IOB-D, IOB-P

/train_dev/train/text_files_iob/train_iob_<.article_id.>.csv